In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
from cursorcommand import handle_cursor_control
from webcommand import handle_gesture_control

# HandTracker
class HandTracker:
    def __init__(self, detectionCon=0.7, maxHands=1):
        self.detectionCon = detectionCon
        self.maxHands = maxHands
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(
            max_num_hands=self.maxHands,
            min_detection_confidence=self.detectionCon,
            min_tracking_confidence=self.detectionCon
        )
        self.mpDraw = mp.solutions.drawing_utils
        self.results = None

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks and draw:
            for handLms in self.results.multi_hand_landmarks:
                self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img

    def getPosition(self, img, handNo=0):
        lmList = []
        if self.results and self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append((cx, cy))
        return lmList

    def getUpFingers(self):
        if self.results and self.results.multi_hand_landmarks:
            hand = self.results.multi_hand_landmarks[0]
            tips = [hand.landmark[i] for i in [4,8,12,16,20]]
            up = [tip.y < hand.landmark[i-2].y for i, tip in zip([4,8,12,16,20], tips)]
            return up
        return [0,0,0,0,0]

# Canvas Button
class ColorRect():
    def __init__(self, x, y, w, h, color, text='', alpha = 0.5):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.color = color
        self.text=text
        self.alpha = alpha

    def drawRect(self, img, text_color=(255,255,255)):
        alpha = self.alpha
        bg_rec = img[self.y:self.y+self.h, self.x:self.x+self.w]
        overlay = np.ones(bg_rec.shape, dtype=np.uint8)*255
        overlay[:] = self.color
        res = cv2.addWeighted(bg_rec, alpha, overlay, 1-alpha, 1.0)
        img[self.y:self.y+self.h, self.x:self.x+self.w] = res
        size = cv2.getTextSize(self.text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
        pos = (int(self.x+self.w/2 - size[0]/2), int(self.y+self.h/2 + size[1]/2))
        cv2.putText(img, self.text, pos, cv2.FONT_HERSHEY_SIMPLEX, 0.8, text_color, 2)

    def isOver(self,x,y):
        return self.x < x < self.x+self.w and self.y < y < self.y+self.h

# Init
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)
detector = HandTracker(detectionCon=0.7)

# Mode control
mode = "cursor"
last_switch_time = 0
switch_cooldown = 1.5
display_text = "Cursor Mode"
last_action_display_time = 0
display_duration = 1.5

# Canvas init
canvas = np.zeros((720,1280,3), np.uint8)
px, py = 0,0
color = (255,0,0)
brushSize = 5
eraserSize = 20
colorsBtn = ColorRect(200,0,100,100,(120,255,0),'Colors')
colors = [ColorRect(400,0,100,100,(0,0,255)), ColorRect(500,0,100,100,(255,0,0)),
          ColorRect(600,0,100,100,(0,255,0)), ColorRect(700,0,100,100,(0,255,255)),
          ColorRect(800,0,100,100,(0,0,0),'Eraser')]
clear = ColorRect(900,0,100,100,(100,100,100),'Clear')
penBtn = ColorRect(1100,0,100,50,color,'Pen')
boardBtn = ColorRect(50,0,100,100,(255,255,0),'Board')
whiteBoard = ColorRect(50,120,1020,580,(255,255,255),alpha=0.6)
hideBoard, hideColors, hidePenSizes = True, True, True
coolingCounter = 20
pens = [ColorRect(1100,50+100*i,100,100,(50,50,50),str(size)) for i,size in enumerate(range(5,25,5))]

# Mode Switch Gestures
def is_L_sign(landmarks):
    index_tip = landmarks[8]
    index_mcp = landmarks[5]
    thumb_tip = landmarks[4]
    thumb_ip = landmarks[3]
    middle_tip = landmarks[12]
    ring_tip = landmarks[16]
    pinky_tip = landmarks[20]

    index_up = index_tip[1] < index_mcp[1]
    thumb_side = abs(thumb_tip[0]-thumb_ip[0])>40
    other_folded = (middle_tip[1]>index_mcp[1] and ring_tip[1]>index_mcp[1] and pinky_tip[1]>index_mcp[1])
    return index_up and thumb_side and other_folded

def is_pinch(landmarks, threshold=40):
    thumb_tip = landmarks[4]
    index_tip = landmarks[8]
    middle_tip = landmarks[12]
    ring_tip = landmarks[16]
    pinky_tip = landmarks[20]

    distance = np.linalg.norm(np.array(thumb_tip) - np.array(index_tip))
    other_folded = (middle_tip[1] > index_tip[1] and ring_tip[1] > index_tip[1] and pinky_tip[1] > index_tip[1])
    return distance < threshold and other_folded

def is_open_palm(upFingers):
    return upFingers == [1,1,1,1,1] 

# Main Loop
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame,1)
    frame = cv2.resize(frame,(1280,720))

    detector.findHands(frame)
    positions = detector.getPosition(frame)
    upFingers = detector.getUpFingers()

    current_time = time.time()
    # Mode Switching 
    if positions:
        if is_L_sign(positions) and mode=="cursor":
            if current_time - last_switch_time > switch_cooldown:
                mode="gesture"
                last_switch_time = current_time
                last_action_display_time = current_time
                display_text = f"Switched to {mode}"
        elif is_pinch(positions) and mode=="gesture":
            if current_time - last_switch_time > switch_cooldown:
                mode="canvas"
                last_switch_time = current_time
                last_action_display_time = current_time
                display_text = f"Switched to {mode}"
                px, py = 0, 0
        elif upFingers and is_open_palm(upFingers) and mode=="canvas":
            if current_time - last_switch_time > switch_cooldown:
                mode="cursor"
                last_switch_time = current_time
                last_action_display_time = current_time
                display_text = f"Switched to {mode}"

    # Mode Logic
    if mode=="cursor":
        action = handle_cursor_control(frame, detector.results)
        if action:
            display_text = action
            last_action_display_time = current_time
        elif current_time - last_action_display_time > display_duration:
            display_text = "Cursor Mode"

    elif mode=="gesture":
        gesture_action = handle_gesture_control(frame, detector.results)
        if gesture_action:
            display_text = f"Gesture Mode: {gesture_action}"
            last_action_display_time = current_time
        elif current_time - last_action_display_time > display_duration:
            display_text = "Gesture Mode"

    elif mode=="canvas":
        if upFingers and positions:
            x, y = positions[8][0], positions[8][1]

            if upFingers[1] and not whiteBoard.isOver(x,y):
                px, py = 0,0
                if not hidePenSizes:
                    for pen in pens:
                        if pen.isOver(x,y):
                            brushSize = int(pen.text)
                            pen.alpha=0
                        else: pen.alpha=0.5
                if not hideColors:
                    for cb in colors:
                        if cb.isOver(x,y):
                            color = cb.color
                            cb.alpha=0
                        else: cb.alpha=0.5
                    if clear.isOver(x,y):
                        canvas = np.zeros((720,1280,3), np.uint8)
                        clear.alpha=0
                    else: clear.alpha=0.5
                if colorsBtn.isOver(x,y) and coolingCounter==0:
                    coolingCounter=10
                    hideColors = not hideColors
                if penBtn.isOver(x,y) and coolingCounter==0:
                    coolingCounter=10
                    hidePenSizes = not hidePenSizes
                if boardBtn.isOver(x,y) and coolingCounter==0:
                    coolingCounter=10
                    hideBoard = not hideBoard
            elif upFingers[1] and not upFingers[2]:
                if whiteBoard.isOver(x,y) and not hideBoard:
                    if px==0 and py==0: px,py = positions[8]
                    if color==(0,0,0):
                        cv2.line(canvas,(px,py),positions[8],color,eraserSize)
                    else:
                        cv2.line(canvas,(px,py),positions[8],color,brushSize)
                    px,py = positions[8]
            else:
                px,py=0,0

            # Draw buttons
            colorsBtn.drawRect(frame)
            boardBtn.drawRect(frame)
            if not hideColors:
                for c in colors: c.drawRect(frame)
                clear.drawRect(frame)
            if not hidePenSizes:
                for p in pens: p.drawRect(frame)
            penBtn.color=color
            penBtn.drawRect(frame)
            # Draw canvas
            if not hideBoard:
                canvasGray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
                _, imgInv = cv2.threshold(canvasGray,20,255,cv2.THRESH_BINARY_INV)
                imgInv = cv2.cvtColor(imgInv, cv2.COLOR_GRAY2BGR)
                frame = cv2.bitwise_and(frame,imgInv)
                frame = cv2.bitwise_or(frame,canvas)

    cv2.putText(frame, display_text,(20,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,255),2)

    display_frame = cv2.resize(frame, (640, 360))  #ukuran tampilan
    cv2.imshow("Hand Control", display_frame)

    if coolingCounter>0: coolingCounter-=1
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1757495798.046178   20393 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1
I0000 00:00:1757495798.062451   20393 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1757495798.092166   22196 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1757495798.092166   22189 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1757495798.115004   22196 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1757495798.115039   22185 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback t